In [29]:
import pandas as pd
import numpy as np
import pathlib
import pickle
from datetime import datetime
import os

In [30]:
Envi = []
BldEnrgy = []

Read energy data for all building files

In [31]:
for path in pathlib.Path("..microclimate_model/Building-Energy").iterdir():
        if path.is_file():
            current_file = pd.read_csv(path)
            BldEnrgy.append(current_file)

Read Envi data for all building files

In [32]:
for path in pathlib.Path("..microclimate_model/ENVI-met/processed data 2").iterdir():
        if path.is_file():
            current_file = pd.read_csv(path)
            Envi.append(current_file)

Rename file names to combine data

In [33]:
for i in range(len(BldEnrgy)):
    # rename column names to combine data
    # file 1
    if BldEnrgy[i]['bldgname'][0] == 'Noble Science Library':
        BldEnrgy[i]['bldgname'] = 'Noble Library'
    # file 2
    if BldEnrgy[i]['bldgname'][0] == 'Bio Design Institute A':
        BldEnrgy[i]['bldgname'] = 'Biodesign A'
    # file 3
    if BldEnrgy[i]['bldgname'][0] == 'Bio Design Institute C':
        BldEnrgy[i]['bldgname'] = 'Biodesign C'
    # file 3
    if BldEnrgy[i]['bldgname'][0] == 'Bio Design Institute B':
        BldEnrgy[i]['bldgname'] = 'Biodesign B'

    print(BldEnrgy[i]['bldgname'][0])

Noble Library
Biodesign A
Biodesign C
Biodesign B
Psychology
Goldwater
Schwada COB
ISTB 2
Bulldog Hall
ISTB 4
Psychology North


In [34]:
for i in range(len(Envi)):
    # file 1
    if Envi[i]['bldgname'][0] == 'Bidesign AB':
        Envi[i]['bldgname'] = 'Biodesign A'
    # file 2
    if Envi[i]['bldgname'][0] == 'Schwada classroom office bldg':
        Envi[i]['bldgname'] = 'Schwada COB'
    # file 3
    if Envi[i]['bldgname'][0] == 'Goldwater Engineering Center':
        Envi[i]['bldgname'] = 'Goldwater'

x = Envi[3].copy()
x['bldgname'] = 'Biodesign B'
Envi.append(x)

for i in range(len(Envi)):
    print(Envi[i]['bldgname'][0])

Rural Parking Structure
Noble Library
Biodesign C
Biodesign A
Schwada COB
Combined Heating and Power Facility
Psychology
Rise-Blakely Hall
Mistake Structure
Skyspace Ramada
LDS Parking garage
Goldwater
LDS Church
ISTB 2
ISTB 7
Tyler Parking Garage
Armstrong Hall
Bulldog Hall
ISTB 4
Psychology North
Biodesign B


Combine data frames

In [35]:
# new list of dataframes
df_list = []
for i in range(len(Envi)):
    for x in range(len(BldEnrgy)):
         if Envi[i]['bldgname'][0] == BldEnrgy[x]['bldgname'][0]:
            result = pd.concat([Envi[i], BldEnrgy[x]], keys=["x", "y"])
            df_list.append(result)

In [36]:
# format ENVI-met time
for i in range(len(df_list)):
    df_list[i]['Time'][['x']] = df_list[i]['Time'][['x']].str.replace('-',':')
    df_list[i]['Time'][['x']] = df_list[i]['Time'][['x']].map(lambda x: str(x)[:-3])
    df_list[i]['Time'][['x']] = pd.to_datetime(df_list[i]['Time'][['x']]).dt.strftime('%H:%M')

# add date and time columns to energy data
for i in range(len(df_list)):
    df_list[i]['Date'][['y']] = df_list[i]['tstamp'][['y']].str[:-13]

for i in range(len(df_list)):
    df_list[i]['Time'][['y']] = df_list[i]['tstamp'][['y']].str[11:16]

for i in range(len(df_list)):
    df_list[i] = df_list[i].drop(columns=['Unnamed: 0', 'tstamp'])


<ipython-input-36-b45bdf76905a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_list[i]['Time'][['x']] = df_list[i]['Time'][['x']].str.replace('-',':')
/Users/alialyakoob/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
<ipython-input-36-b45bdf76905a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_list[i]['Time

In [37]:
new_list = []
for i in range(len(df_list)):
    x = df_list[i].loc[('x')].copy()
    x['Date_Time'] = x['Date'] + ' ' + x['Time']
    x.Date_Time = pd.to_datetime(x.Date_Time).dt.strftime('%m-%d-%Y %H:%M')
    # x = x.set_index('Date_Time')
    # x = x.drop(columns = ['Date', 'Time'])
    y = df_list[i].loc[('y')].copy()
    y = y.drop(columns = ['bldgname', 'Air Temp', 'Rel Hum'])
    y['Date_Time'] = y['Date'] + ' ' + y['Time']
    y.Date_Time = pd.to_datetime(y.Date_Time).dt.strftime('%m-%d-%Y %H:%M')
    y = y.set_index('Date_Time')
    y = y.drop(columns = ['Date', 'Time'])
    # change date_time of data with 23:59 to next day with 00:00
    x['Date_Time'][x['Date_Time'].str.contains("05-16-2018 23:59")] = '05-17-2018 00:00'
    x['Date_Time'][x['Date_Time'].str.contains("05-23-2018 23:59")] = '05-17-2018 00:00'
    x['Date_Time'][x['Date_Time'].str.contains("06-07-2018 23:59")] = '06-08-2018 00:00'
    x['Date_Time'][x['Date_Time'].str.contains("06-08-2018 23:59")] = '06-09-2018 00:00'
    x['Date_Time'][x['Date_Time'].str.contains("06-20-2018 23:59")] = '06-21-2018 00:00'
    x['Date_Time'][x['Date_Time'].str.contains("06-21-2018 23:59")] = '06-21-2018 00:00'
    x['Date_Time'][x['Date_Time'].str.contains("06-25-2018 23:59")] = '06-26-2018 00:00'
    x['Date_Time'][x['Date_Time'].str.contains("08-03-2018 23:59")] = '05-17-2018 00:00'
    x['Date_Time'][x['Date_Time'].str.contains("09-11-2018 23:59")] = '09-12-2018 00:00'
    x['Date_Time'][x['Date_Time'].str.contains("09-29-2018 23:59")] = '09-30-2018 00:00'
    for i in range(len(x)):
        try:
            x['CHWTON'][i] = y['CHWTON'].loc[x['Date_Time'][i]]
            x['KW'][i] = y['KW'].loc[x['Date_Time'][i]]
            x['HTmmBTU'][i] = y['HTmmBTU'].loc[x['Date_Time'][i]]
        except KeyError:
            pass

    new_list.append(x)

<ipython-input-37-7b130471c633>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['Date_Time'][x['Date_Time'].str.contains("05-16-2018 23:59")] = '05-17-2018 00:00'
<ipython-input-37-7b130471c633>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['Date_Time'][x['Date_Time'].str.contains("05-23-2018 23:59")] = '05-17-2018 00:00'
<ipython-input-37-7b130471c633>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['Date_Time'][x['Date_

In [38]:
# check if any time format is in 23:59
x['Date_Time'][x['Date_Time'].str.contains("23:59")]

Series([], Name: Date_Time, dtype: object)

In [39]:
x

,bldgname,Date,Time,Air Temp,Rel Hum,KW,CHWTON,HTmmBTU,Date_Time
0,Biodesign B,2018-05-16,05:00,24.040925,20.616757,457.98,334.01,0.58,05-16-2018 05:00
1,Biodesign B,2018-05-16,05:15,24.220021,23.379832,461.70,260.40,0.62,05-16-2018 05:15
2,Biodesign B,2018-05-16,05:30,24.054303,23.070921,458.10,268.27,0.62,05-16-2018 05:30
3,Biodesign B,2018-05-16,05:45,23.935568,23.126250,466.81,271.76,0.61,05-16-2018 05:45
4,Biodesign B,2018-05-16,06:00,23.808253,23.394283,455.00,270.87,0.58,05-16-2018 06:00
5,Biodesign B,2018-05-16,06:15,23.627885,25.141569,464.68,247.19,0.56,05-16-2018 06:15
6,Biodesign B,2018-05-16,06:30,23.752276,25.699406,466.26,285.06,0.64,05-16-2018 06:30
7,Biodesign B,2018-05-16,06:45,24.085777,25.783490,473.61,297.82,0.62,05-16-2018 06:45
8,Biodesign B,2018-05-16,07:00,24.508369,25.428487,492.91,312.38,0.60,05-16-2018 07:00
9,Biodesign B,2018-05-16,07:15,25.059040,25.016651,507.23,364.44,0.70,05-16-2018 07:15


In [40]:
pd.set_option('display.max_rows', None)
for i in range(len(new_list)):
    print(len(new_list[i]))

770
770
770
770
770
770
770
770
770
770
770


In [41]:
# save combined energy and ENVI data
for i in range(len(new_list)):
    new_list[i].to_csv("..microclimate_model/Combined/dataset2/" + new_list[i]['bldgname'][0] + ".csv", index=True)